### 1º Voy a priorizar en primer lugar a los  diseñadores y desarrolladores (lugares con aglomeración de empresas de videojuegos y tecnológicas que hayan facturado más de 1M en su historia, las de videojuegos cumplen el perfil en este caso)

Me quedo de la base de datos Companies con las empresas de videojuegos que han facturado más de 1M, ejecuto el script que escribe el json con la consulta filtrando la base de datos todo con el formato adecuado

In [1]:
from getJsonGames1M import writeJsonOffice
writeJsonOffice()

Hago el mongoimport e importo la colección a la base de datos,indexo el campo position como índice geoespacial. Ahora puedo buscar los lugares donde se aglomeran estas empresas, y buscar la lista de ciudades. El scrip devuelve una lista de ciudades donde hay empresas que a 100km a la redonda tienen más de 5 empresas de videojuegos que han facturado más de 1M, creo que es suficiente para satisfacer a los desarrolladores y a los diseñadores. 

In [2]:
from getCitiesDevNDesigner import citiesDevNDesignerOK
citiesDevNDes = citiesDevNDesignerOK()

In [3]:
len(citiesDevNDes)


45

### 2º En las anteriores ciudades voy a buscar qué tal son las comunicaciones, así cumplo con el requisito de los ejecutivos, para ello ejecuto el script que buscará aeropuertos, estaciones de tren y autobús con la api de google place pasándole el diccionario de las ciudades con sus coordenadas. El criterio va a ser que al menos haya 1 aeropuerto y 10 estaciones de tren o metro en 30 km a la redonda. Es de suponer que todas las ciudades que cumplan el criterio también tendrán buenas comunicaciones por carretera.

In [4]:
from filterCitiesForExecutive import getCities

In [5]:
cities = getCities(citiesDevNDes)

In [6]:
len(cities)

37

### 3º Ahora voy a buscar ciudades con una buena aglomeración de Starbucks, pienso que 5 en 10 km a la redonda es adecuado

In [7]:
from filterCitiesStarbucks import filterStarbucks

In [8]:
cities = list(filter(filterStarbucks,cities))

In [9]:
len(cities)

37

### 4º El siguiente filtro que voy a aplicar a mi lista es uno que satisfaga a las familias, me ha parecido que el criterio de que haya más de 3 lugares con la palabra universidad en 10 km cuadrados es un buen criterio, si hay universidades también hay colegios y guarderías para los niños

In [10]:
from filterCitiesSchool import filterUniversity
cities = list(filter(filterUniversity,cities))


In [11]:
len(cities)


35

### 5º Ahora voy a buscar que haya una buenna aglomeración de bares en la ciudad, si hay bares habrá discotecas y demás establecimientos de ocio nocturno y diurno.40 bares en 10 km

In [12]:
from filterCitiesBar import filterBar
cities = list(filter(filterBar,cities))

In [13]:
len(cities)

35

### 6º Ciudades veganas, mas de 10 sitios que contengan 'vegan' en 10km a la redonda

In [25]:
from filterCitiesBar import filterVegan
cities = list(filter(filterVegan,cities))
len(cities)

[{'city': 'New York',
  'lat': 40.735781556250004,
  'long': -73.97903525000001,
  'rentIndex': 100.0},
 {'city': 'Brooklyn',
  'lat': 40.714952833333335,
  'long': -73.9684315,
  'rentIndex': 80.39},
 {'city': 'NYC', 'lat': 40.801358, 'long': -74.3372},
 {'city': 'San Francisco',
  'lat': 37.75123599565217,
  'long': -122.38115247826084,
  'rentIndex': 114.06},
 {'city': 'Mountain View',
  'lat': 37.39952858333333,
  'long': -122.06723616666666},
 {'city': 'San Mateo', 'lat': 37.514257480000005, 'long': -122.25373152},
 {'city': 'Redwood City', 'lat': 37.474702666666666, 'long': -122.216019},
 {'city': 'Palo Alto', 'lat': 37.4297035, 'long': -122.1542725},
 {'city': 'Sunnyvale', 'lat': 37.393121550000004, 'long': -122.00949084999999},
 {'city': 'Singapore',
  'lat': 37.580304,
  'long': -122.343679,
  'rentIndex': 61.99},
 {'city': 'San Bruno', 'lat': 37.627971, 'long': -122.426804},
 {'city': 'Menlo Park', 'lat': 37.41924, 'long': -122.208414},
 {'city': 'Los Angeles',
  'lat': 37.44

In [26]:
pd.DataFrame(cities).to_json('../outputs/citiesOk.json')

In [5]:
import pandas as pd
citiesOk = list(pd.read_json('../outputs/citiesOk.json').T.to_dict().values())

### 7. Hago un scraping para  añadir a cada ciudad su índice de coste de vida, me quedaré con el más bajo, creo que eso es bueno tanto para los trabajadores como para la empresa, porque ese índice incluye la adquisición de bienes y servicios.

In [7]:
from getIndex import getRent
import pandas as pd
pd.DataFrame(getRent(citiesOk)).sort_values(by ='rentIndex' ).reset_index().drop(columns = 'index').to_csv('../outputs/okByRent.csv')


In [1]:
import pandas as pd
pd.read_csv('../outputs/okByRent.csv').drop(columns = 'Unnamed: 0',index = 0)

,city,lat,long,rentIndex
1,Philadelphia,39.952576,-75.165252,46.51
2,Singapore,37.580304,-122.343679,61.99
3,Los Angeles,37.446823,-122.161523,72.16
4,Boston,42.349149,-71.062173,73.94
5,Brooklyn,40.714953,-73.968431,80.39
6,New York,40.735782,-73.979035,100.00
7,San Francisco,37.751236,-122.381152,114.06
8,NYC,40.801358,-74.337200,NaN
9,Mountain View,37.399529,-122.067236,NaN
10,San Mateo,37.514257,-122.253732,NaN


#### Mi objetivo con esto es quedarme con la ciudad de renta más baja, solo porque supongo que a la empresa es lo que le conviene, una ciudad que dentro de que cumpla los requisitos sea lo más barata posible en temas de sueldos.

### 8. Obtengo todas la compañías de Philadelphia con el formato adecuado para indexar

In [2]:
from getJsonPhiladelphia import writeJson
writeJson()

Importo el Json como una nueva colección a mi base de datos, sobre esa colección elaboraré la solución final.


In [ ]:
def officeCambridgeOk():
    from pymongo import MongoClient
    import pandas as pd
    from getMongoClient import getClient
    from getEnv import getVariable
    client = getClient()
    dbCompanies = client.companies
    cambridge1m = list(dbCompanies.cambridge.find())
    listCambridge = getListOffice2km(cambridge1m,dbCompanies)


    #listGame1M100K=list(filter(lambda df:True if df.name.count() in range(5,maximo+1) else False,listGame1M100K))

    return listCambridge


def getCompaniesNear(lat,long, max_Distance,db):
    return list(db.cambridge.find({
        "position": {
            "$near": {
               "$geometry": {
                  "type": "Point" ,
                  "coordinates": [ long , lat ]
               },
               "$maxDistance": max_Distance
               
             }
       }
    }))

def getListOffice2km(listOffice,db):
    import pandas as pd
    listDf = []
    for ele in listOffice:
        if ele['founded_year'] > 2008:
            print(ele)
            df = pd.DataFrame(getCompaniesNear(ele['lat'],ele['long'],2000,db))
            listDf.append(df.reset_index().drop(columns=['index','_id','position']))
    return listDf

In [ ]:
l = officeCambridgeOk()
len(l)

In [ ]:
import pandas as pd
pd.DataFrame(l[1])

In [ ]:
import plotly.express as px
import pandas as pd
data = pd.read_json('../outputs/cambridgeOffices.json')
fig = px.scatter_mapbox(data, lat="lat", lon="long", hover_name='name', hover_data=["category"],
                        color_discrete_sequence=["fuchsia"], zoom=1, height=700)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
fig.show()
